In [123]:
import pandas as pd
import torch
import numpy as np
import torch.utils.data
import torch.nn as nn
from pandas import read_csv
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

In [124]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

data = load_file('wifi_localization.txt')

dfd = pd.DataFrame(data)

df=dfd.sample(frac=1)
fold_size=400


def getdf(i):
    testdf=df[(fold_size*i):(fold_size*i+fold_size)]
    traindf=pd.concat([df, testdf,testdf]).drop_duplicates(keep=False)
    return traindf,testdf

def get_data(traindf,testdf):
    
    colname = traindf.columns[df.shape[1]-1]
    X_train=traindf.drop(colname , axis=1).values
    y_train=traindf[colname]


    colname = testdf.columns[df.shape[1]-1]
    X_test=testdf.drop(colname , axis=1).values
    y_test=testdf[colname]


    X_train = torch.from_numpy(X_train)
    X_test=torch.from_numpy(X_test)

    y_train=torch.Tensor(np.array(y_train))
    y_test=torch.Tensor(np.array(y_test))



    for i in range(0,4):
        y_train[y_train==i+1]=i
        y_test[y_test==i+1]=i
    y_truee=y_test
    
    return X_train,X_test,y_train,y_test,y_truee


In [125]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(7, 50)
        self.l2 = nn.Linear(50, 25)
        self.l3 = nn.Linear(25, 4)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        return self.l3(x)


model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


In [126]:


def train(X_train,y_train):
    
    for i in range(0,l):

        X_t=X_train[(batch_size*i):(batch_size*i+batch_size)]
        y_t=y_train[(batch_size*i):(batch_size*i+batch_size)]    

        optimizer.zero_grad()
        output=model(X_t)
        loss = criterion(output, y_t)
        loss.backward()
        optimizer.step()



In [127]:



def test(X_test,y_test):
    counter=1
    test_loss=0

    X_test=X_test.float()
    y_test=y_test.long()

    model.eval()

    temp=torch.tensor([])
    y_true=y_truee.cpu().detach().numpy()


    output=model(X_test)

    q=F.softmax(output,dim=1)
    qa=F.softmax(q,dim=1)
    aba=qa.cpu().detach().numpy()
    y_pred=np.argmax(aba,axis=1)
    acc=accuracy_score(y_test,y_pred)
    recall=recall_score(y_test,y_pred,average='macro')
    prec=precision_score(y_test,y_pred,average='macro')
    f1=f1_score(y_test,y_pred,average='macro')
    return acc,prec,recall,f1
    #print("Acc: %3.5f, P: %3.5f, R: %3.5f, f1_score: %3.5f" % (acc, prec, recall,f1))

        

accu=0
reca=0
preci=0
f1s=0
    
for t in range(0,5):
    traindf,testdf=getdf(t)
    X_train,X_test,y_train,y_test,y_truee=get_data(traindf,testdf)
    counter=0
    batch_size=50
    lenth=len(X_train)/batch_size
    l=int(lenth)
    X_train=X_train.float()
    y_train=y_train.long()
    model.train()
    
    for i in range(0,40):
        train(X_train,y_train)
        
    acc,prec,recall,f1=test(X_test,y_test)
    print(t+1,"fold= Acc: %3.5f, P: %3.5f, R: %3.5f, f1_score: %3.5f" % (acc, prec, recall,f1))
    accu+=acc
    preci+=prec
    reca+=recall
    f1s+=f1

print('\n')
print("Average Acc: %3.5f, P: %3.5f, R: %3.5f, f1_score: %3.5f" % (accu/5, preci/5, reca/5,f1s/5))

1 fold= Acc: 0.92750, P: 0.93695, R: 0.93483, f1_score: 0.93062
2 fold= Acc: 0.95000, P: 0.95172, R: 0.95003, f1_score: 0.94891
3 fold= Acc: 0.97000, P: 0.97122, R: 0.96820, f1_score: 0.96944
4 fold= Acc: 0.97500, P: 0.97694, R: 0.97334, f1_score: 0.97454
5 fold= Acc: 0.96500, P: 0.96875, R: 0.96819, f1_score: 0.96668


Average Acc: 0.95750, P: 0.96112, R: 0.95892, f1_score: 0.95804
